<a href="https://colab.research.google.com/github/VEDANTPARAB404/SY-SKILL/blob/main/LCA3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

file1 = '/content/drive/My Drive/archive (9)/reduced_data_1.csv'
file2 = '/content/drive/My Drive/archive (9)/reduced_data_2.csv'
file3 = '/content/drive/My Drive/archive (9)/reduced_data_3.csv'
file4 = '/content/drive/My Drive/archive (9)/reduced_data_4.csv'
import pandas as pd

df1 = pd.read_csv(file1)
df2 = pd.read_csv(file2)
df3 = pd.read_csv(file3)
df4 = pd.read_csv(file4)

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


<ipython-input-1-5c64fd4022d1>:10: DtypeWarning: Columns (7,9) have mixed types. Specify dtype option on import or set low_memory=False.
  df1 = pd.read_csv(file1)


In [ ]:
df_main=pd.concat([df1,df2,df3,df4])
df_main.head()

In [ ]:
df_main.isnull().sum()

In [ ]:
df_main.isna().sum()

In [ ]:
df_main.drop_duplicates(inplace=True)

In [ ]:
df_main.info()

Droping useless features to improve the accuracy

In [ ]:
drop=['pkSeqID','subcategory','category']
df_main.drop(columns=drop, inplace=True, errors='ignore')

print("New Columns: ")
df_main.columns

Label Encoding

In [ ]:
from sklearn.preprocessing import LabelEncoder

label_encoders = {}
for column in df_main.select_dtypes(include=['object', 'category']).columns:
    df_main[column] = df_main[column].astype(str)  # Ensure all values are strings
    le = LabelEncoder()
    df_main[column] = le.fit_transform(df_main[column])
    label_encoders[column] = le

print("✅complete")
print(df_main.head())

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns

correlation_matrix = df_main.corr()

plt.figure(figsize=(35, 35))  # Adjust figure size if needed
sns.heatmap(correlation_matrix, annot=True, cmap='coolwarm', fmt=".2f")
plt.title('Correlation Matrix Heatmap')
plt.show()

In [ ]:
attack_counts = df_main['attack'].value_counts()
plt.figure(figsize=(10, 6))  # Adjust figure size if needed
sns.countplot(x='attack', data=df_main, order=attack_counts.index)
plt.title('Distribution of Attack Types')
plt.xlabel('Attack Type')
plt.ylabel('Count')
plt.xticks(rotation=45, ha='right')  # Rotate x-axis labels if needed
plt.show()

**Balancing**(to prevent bias)

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.utils import resample

target = "attack"

# 1. Original X and y
X = df_main.drop(target, axis=1)
y = df_main[target]

# 2. Train-test split FIRST (important!)
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, stratify=y, random_state=42
)

# 3. Balance the training set ONLY
train_df = pd.concat([X_train, y_train], axis=1)
grouped = [g for _, g in train_df.groupby(target)]
balanced_train = [
    resample(g, replace=True, n_samples=100000, random_state=42)  # or any target size
    for g in grouped
]
balanced_train_df = pd.concat(balanced_train).sample(frac=1, random_state=42)

# 4. Now separate X and y again
X_train_bal = balanced_train_df.drop(target, axis=1)
y_train_bal = balanced_train_df[target]


print("Balanced Training Set Shape:", X_train_bal.shape, y_train_bal.shape)



Feature Scaling

In [ ]:
from sklearn.preprocessing import StandardScaler
import pandas as pd

# Use the balanced training set from your updated code
X = X_train_bal
y = y_train_bal

# Scale the features
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

# (Optional) Convert back to DataFrame for inspection
X_scaled_df = pd.DataFrame(X_scaled, columns=X.columns)

print("✅ Feature scaling done")
print(X_scaled_df.head())


Exploratory Data Analysis (EDA)

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns

# Combine X_train_bal and y_train_bal temporarily for correlation
temp_df = X_train_bal.copy()
temp_df["attack"] = y_train_bal

# Plot correlation matrix
plt.figure(figsize=(12, 8))
corr = temp_df.corr()
sns.heatmap(corr, cmap='coolwarm', annot=False, linewidths=0.5)
plt.title("Feature Correlation Heatmap")
plt.show()


In [ ]:
import matplotlib.pyplot as plt

# Count of each attack type
attack_counts = balanced_train_df['attack'].value_counts()

# Plot
plt.figure(figsize=(8, 8))
plt.pie(attack_counts, labels=attack_counts.index, autopct='%1.1f%%', startangle=140, colors=plt.cm.tab20.colors)
plt.title('Distribution of Attack Types (Balanced Training Set)')
plt.axis('equal')  # Equal aspect ratio ensures pie is drawn as a circle
plt.show()


**Feature** **Selection**

FILTER METHODS

In [ ]:
#ANOVA

from sklearn.feature_selection import f_classif, SelectKBest
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.utils import resample
import pandas as pd

target = "attack"

# Step 1: Split original data
X = df_main.drop(target, axis=1)
y = df_main[target]

X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, stratify=y, random_state=42
)

# Step 2: Balance training data only
train_df = pd.concat([X_train, y_train], axis=1)
grouped = [g for _, g in train_df.groupby(target)]
balanced_train = [
    resample(g, replace=True, n_samples=100000, random_state=42)
    for g in grouped
]
balanced_train_df = pd.concat(balanced_train).sample(frac=1, random_state=42)

X_train_bal = balanced_train_df.drop(target, axis=1)
y_train_bal = balanced_train_df[target]

# Step 3: Apply ANOVA on training set
selector = SelectKBest(score_func=f_classif, k=10)
X_train_selected = selector.fit_transform(X_train_bal, y_train_bal)

selected_feature_indices = selector.get_support(indices=True)
selected_features = X_train_bal.columns[selected_feature_indices]
print("Selected features:", list(selected_features))

# Step 4: Transform test set with same features
X_test_selected = X_test[selected_features]

# Step 5: Train model and evaluate
model = RandomForestClassifier(random_state=42)
model.fit(X_train_selected, y_train_bal)
y_pred = model.predict(X_test_selected)

accuracy = accuracy_score(y_test, y_pred)
print(f"Accuracy of the model: {accuracy:.4f}")


In [ ]:
#chi2

from sklearn.feature_selection import chi2

# Assuming X_train_bal and y_train_bal are already defined from the previous code

# Apply Chi2
selector = SelectKBest(score_func=chi2, k=10)  # Select top 10 features
X_train_selected = selector.fit_transform(X_train_bal, y_train_bal)

selected_feature_indices = selector.get_support(indices=True)
selected_features = X_train_bal.columns[selected_feature_indices]
print("Selected features (Chi2):", list(selected_features))

# Transform the test set using the selected features
X_test_selected = X_test[selected_features]

# Train a RandomForestClassifier
model = RandomForestClassifier(random_state=42)
model.fit(X_train_selected, y_train_bal)
y_pred = model.predict(X_test_selected)

# Calculate accuracy
accuracy = accuracy_score(y_test, y_pred)
print(f"Accuracy of the model (Chi2): {accuracy:.4f}")


In [ ]:
# Correlation-based feature selection
correlation_threshold = 0.7

# Calculate the correlation matrix for the training data (X_train_bal)
correlation_matrix = X_train_bal.corr()

# Find highly correlated features
correlated_features = set()
for i in range(len(correlation_matrix.columns)):
    for j in range(i):
        if abs(correlation_matrix.iloc[i, j]) > correlation_threshold:
            colname = correlation_matrix.columns[i]
            correlated_features.add(colname)

# Get all features present in both training and test data
all_features = list(set(X_train_bal.columns) & set(X_test.columns))

# Remove correlated features from the list of all features
selected_features = [feature for feature in all_features if feature not in correlated_features]

# Use selected features for training and test data
X_train_corr = X_train_bal[selected_features]
X_test_corr = X_test[selected_features]

# Train RandomForestClassifier with selected features
model = RandomForestClassifier(random_state=42)
model.fit(X_train_corr, y_train_bal)

# Predict on test data
y_pred_corr = model.predict(X_test_corr)

print("Selected Featues: ",selected_features)

# Calculate accuracy
accuracy_corr = accuracy_score(y_test, y_pred_corr)
print(f"Accuracy with Correlation-based feature selection: {accuracy_corr:.4f}")

**WRAPPER** **METHODS**

In [ ]:
# ffs wrapper to find the features and accuracy

from sklearn.feature_selection import SequentialFeatureSelector

# Sequential Feature Selection (SFS)
sfs = SequentialFeatureSelector(RandomForestClassifier(random_state=42), n_features_to_select=10, direction='forward')
X_train_sfs = sfs.fit_transform(X_train_bal, y_train_bal)

selected_feature_indices_sfs = sfs.get_support(indices=True)
selected_features_sfs = X_train_bal.columns[selected_feature_indices_sfs]
print("Selected features (SFS):", list(selected_features_sfs))

# Transform the test set using the selected features
X_test_sfs = X_test[selected_features_sfs]

# Train a RandomForestClassifier with selected features
model_sfs = RandomForestClassifier(random_state=42)
model_sfs.fit(X_train_sfs, y_train_bal)
y_pred_sfs = model_sfs.predict(X_test_sfs)

# Calculate accuracy
accuracy_sfs = accuracy_score(y_test, y_pred_sfs)
print(f"Accuracy of the model (SFS): {accuracy_sfs:.4f}")


# Sequential Feature Selection (SBS)
sbs = SequentialFeatureSelector(RandomForestClassifier(random_state=42), n_features_to_select=10, direction='backward')
X_train_sbs = sbs.fit_transform(X_train_bal, y_train_bal)

selected_feature_indices_sbs = sbs.get_support(indices=True)
selected_features_sbs = list(X_train_bal.columns[selected_feature_indices_sbs])

print("Selected features (SBS):", selected_features_sbs)

# Transform the test data
X_test_sbs = X_test[selected_features_sbs]

# Train a model and evaluate
model_sbs = RandomForestClassifier(random_state=42)
model_sbs.fit(X_train_sbs, y_train_bal)
y_pred_sbs = model_sbs.predict(X_test_sbs)
accuracy_sbs = accuracy_score(y_test, y_pred_sbs)

print(f"Accuracy of the model (SBS): {accuracy_sbs:.4f}")


In [ ]:
from sklearn.feature_selection import RFE, SelectKBest, f_classif
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.utils import resample


target = "attack"

# 1. Original X and y
X = df_main.drop(target, axis=1)
y = df_main[target]

# 2. Train-test split FIRST (important!)
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, stratify=y, random_state=42
)

# 3. Balance the training set ONLY
train_df = pd.concat([X_train, y_train], axis=1)
grouped = [g for _, g in train_df.groupby(target)]
balanced_train = [
    resample(g, replace=True, n_samples=100000, random_state=42)
    for g in grouped
]
balanced_train_df = pd.concat(balanced_train).sample(frac=1, random_state=42)

# 4. Now separate X and y again
X_train_bal = balanced_train_df.drop(target, axis=1)
y_train_bal = balanced_train_df[target]

# 1. Pre-filter features using ANOVA (select top 20)
pre_selector = SelectKBest(f_classif, k=20)
X_train_prefiltered = pre_selector.fit_transform(X_train_bal, y_train_bal)
X_test_prefiltered = pre_selector.transform(X_test)

# 2. Initialize a simpler model (or reduce complexity of RandomForest)
model = RandomForestClassifier(n_estimators=50, random_state=42)

# 3. Backward Feature Elimination with optimizations
num_features = X_train_prefiltered.shape[1]  # Start with fewer features
best_features = []
best_accuracy = 0

for i in range(num_features, 0, -2):  # Increase step size to 2 (or more)
    selector = RFE(estimator=model, n_features_to_select=i, step=2, n_jobs=-1)  # Parallelize
    selector = selector.fit(X_train_prefiltered, y_train_bal)
    selected_features = X_train_bal.columns[pre_selector.get_support(indices=True)][selector.support_]

    X_train_rfe = selector.transform(X_train_prefiltered)
    X_test_rfe = selector.transform(X_test_prefiltered)

    model_final = RandomForestClassifier(random_state=42) # Use your original model here if desired
    model_final.fit(X_train_rfe, y_train_bal)
    y_pred = model_final.predict(X_test_rfe)
    accuracy = accuracy_score(y_test, y_pred)

    if accuracy > best_accuracy:
        best_accuracy = accuracy
        best_features = list(selected_features)

    print(f"Number of features: {i}, Accuracy: {accuracy:.4f}")

print(f"\nBest Number of Features: {len(best_features)}")
print(f"Best Accuracy: {best_accuracy:.4f}")
print(f"Best Features: {best_features}")

EMBEDDED

In [ ]:

# Embedded Method Example (L1 Regularization with Logistic Regression)
from sklearn.linear_model import LogisticRegression

# Use the selected features from previous steps (e.g., ANOVA)
X_selected = X[selected_features]

# Split data
X_train, X_test, y_train, y_test = train_test_split(
    X_selected, y, test_size=0.2, random_state=42
)

# Initialize and train the model with L1 penalty
model = LogisticRegression(penalty='l1', solver='liblinear', random_state=42) # Use liblinear solver for L1
model.fit(X_train, y_train)

# Make predictions
y_pred_embedded = model.predict(X_test)

# Evaluate the model
accuracy_embedded = accuracy_score(y_test, y_pred_embedded)
print(f"Accuracy with Embedded Method (L1 Regularization): {accuracy_embedded}")

# You can experiment with other models that have built-in feature selection,
# such as decision trees or support vector machines with L1 regularization.


**Bagging**

In [ ]:
# bagging

from sklearn.ensemble import BaggingClassifier

# Assuming X_train_bal, y_train_bal, X_test, and y_test are defined from previous code

# Initialize the base estimator (e.g., decision tree)
base_estimator = RandomForestClassifier(random_state=42)

# Initialize the BaggingClassifier
bagging_model = BaggingClassifier(estimator=base_estimator, n_estimators=10, random_state=42)

# Train the bagging model
bagging_model.fit(X_train_bal, y_train_bal)

# Make predictions on the test set
y_pred_bagging = bagging_model.predict(X_test)

# Evaluate the bagging model
accuracy_bagging = accuracy_score(y_test, y_pred_bagging)
print(f"Accuracy of the Bagging model: {accuracy_bagging:.4f}")


Boosting

In [ ]:
#boosting

from sklearn.ensemble import AdaBoostClassifier, GradientBoostingClassifier

# Initialize the AdaBoostClassifier
adaboost_model = AdaBoostClassifier(n_estimators=50, random_state=42)  # Adjust n_estimators as needed

# Train the AdaBoost model
adaboost_model.fit(X_train_bal, y_train_bal)

# Make predictions on the test set
y_pred_adaboost = adaboost_model.predict(X_test)

# Evaluate the AdaBoost model
accuracy_adaboost = accuracy_score(y_test, y_pred_adaboost)
print(f"Accuracy of the AdaBoost model: {accuracy_adaboost:.4f}")


# Initialize the GradientBoostingClassifier
gb_model = GradientBoostingClassifier(n_estimators=100, random_state=42)  # Adjust n_estimators as needed

# Train the Gradient Boosting model
gb_model.fit(X_train_bal, y_train_bal)

# Make predictions
y_pred_gb = gb_model.predict(X_test)

# Evaluate the model
accuracy_gb = accuracy_score(y_test, y_pred_gb)
print(f"Accuracy of the Gradient Boosting model: {accuracy_gb:.4f}")


In [ ]:
# create a graph for all the accuracy comparison of the feature selection weve done

import matplotlib.pyplot as plt

# Accuracy scores (replace with your actual values)
accuracy_scores = {
    'ANOVA': 1,  # Replace with your ANOVA accuracy
    'Chi2': 1,  # Replace with your Chi2 accuracy
    'Correlation': 1,  # Replace with your correlation accuracy
    'RFE': 1,  # Replace with your RFE accuracy
    'Backward Elimination': 1.00,  # Replace with your Backward Elimination accuracy
    'Embedded (L1)': 0.99,  # Replace with your Embedded method accuracy
    'Bagging': 1,  # Replace with your Bagging accuracy
    'AdaBoost': 0.985, # Replace with your AdaBoost accuracy
    'GradientBoosting':0.99 #Replace with your Gradient Boosting accuracy
}

# Method names
methods = list(accuracy_scores.keys())

# Accuracy values
scores = list(accuracy_scores.values())


plt.figure(figsize=(10, 6))  # Adjust figure size if needed
plt.bar(methods, scores, color='red')
plt.xlabel("Feature Selection Methods")
plt.ylabel("Accuracy")
plt.title("Accuracy Comparison of Feature Selection Methods")
plt.xticks(rotation=45, ha="right")  # Rotate x-axis labels for better readability
plt.tight_layout()  # Adjust layout to prevent labels from overlapping
plt.show()
